In [4]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import time

# Data preprocessing (only once)

In [1]:
# contains available classes in this data

with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\wnids.txt" , "r") as f:
    classes = []
    for line in f :
        line = line.removesuffix("\n")
        classes.append(line)



# contains all classes in old file
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\words.txt" , "r") as f:
    labels =[]
    for line in f :
        line = line.removesuffix("\n")
        line = line.split("\t")
        
        labels.append(line)
        


### make file with names of classes in subset (only once)

In [2]:
existing = [] 
for file , label in labels:
    if file in classes:
        existing.append(f"{file}-{label}\n")
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\tiny_file-class","+w") as f:
    f.writelines(existing[0:100])

# Image import 

In [5]:
files_cls = []
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\tiny_file-class" , "r") as f :
    for line in f :
        line = line.strip()
        
        file_name , labels = line.split("-", maxsplit=1)
        labels = labels.split(",")
        files_cls.append([file_name, labels])


In [6]:
import os

train_dir = r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\train-100"
# uses second half


image_data = []

for root, dirs, files in os.walk(train_dir):
    for file in files:
        if file.endswith((".jpg", ".JPEG", ".png")):
            file_path = os.path.join(root, file)
 
            label = os.path.basename(os.path.dirname(root))
            
            image_data.append((file_path, label))
            

# Print some of the data

print("Number of images:", len(image_data))

file_path # path to each image
label # label code of each image


Number of images: 50000


'n03250847'

In [9]:
cv_images = []
labels = []
# make it np
i = 1
num = len(image_data)
start_t = time.time()
for name , cls in image_data:
    image = cv2.imread(name)
    labels.append(cls)
    cv_images.append(image)
    
    if i%200 == 0:
        print(f"{i}/{num}")
        # break #############################
    i=i+1
    
cv_images = np.array(cv_images)
labels = np.array(labels)


print(time.time()-start_t)

200/50000
400/50000
600/50000
800/50000
1000/50000
1200/50000
1400/50000
1600/50000
1800/50000
2000/50000
2200/50000
2400/50000
2600/50000
2800/50000
3000/50000
3200/50000
3400/50000
3600/50000


KeyboardInterrupt: 

# feature extraction

In [ ]:
from skimage.feature import hog , local_binary_pattern

# HOG 
def extract_hog(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray_image, orientations=9, pixels_per_cell=(8,8),
                      cells_per_block=(2, 2), block_norm='L2-Hys',
                      visualize=True, feature_vector=True)
    return features

# LBP 
def extract_lbp(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)
    return lbp_hist

# Color 

def extract_color_hist(image):
    hist_red = cv2.calcHist([image], [0], None, [32], [0, 256])
    hist_green = cv2.calcHist([image], [1], None, [32], [0, 256])
    hist_blue = cv2.calcHist([image], [2], None, [32], [0, 256])
    color_hist = np.concatenate([hist_red, hist_green, hist_blue]).flatten()
    return color_hist / color_hist.sum()  # Normalize


# reduce size for trial

# cv_images = cv_images[:10000]
# labels = labels[:10]

hog_features = []
lbp_features = []
color_hist_features = []
lbp_features64 = []
color_hist_features64 = []




i = 1
for image in cv_images:
    if image is None:  # Skip invalid images
        continue

    # Resize the image to a fixed size for consistency
    image_resized = cv2.resize(image, (128, 128))


    # here we increased resolution for ibp and color to get better results as their computation cost is quite small
    hog_feat = extract_hog(image)
    lbp_feat64 = extract_lbp(image)
    color_feat64 = extract_color_hist(image)
    
    lbp_feat = extract_lbp(image_resized)
    color_feat = extract_color_hist(image_resized)
    
    # Append to respective lists
    hog_features.append(hog_feat)
    lbp_features.append(lbp_feat)
    color_hist_features.append(color_feat)
    
    lbp_features64.append(lbp_feat64)
    color_hist_features64.append(color_feat64)
    
    if i % 400 == 0:
        print(f"Processed {i} images")
    i += 1

# Convert lists to NumPy arrays for further processing
hog_features = np.array(hog_features)
lbp_features = np.array(lbp_features)
color_hist_features = np.array(color_hist_features)
lbp_features64 = np.array(lbp_features64)
color_hist_features64 = np.array(color_hist_features64)


print(f"HOG features: {hog_features.shape}")
print(f"LBP features: {lbp_features.shape}")
print(f"Color Histogram features: {color_hist_features.shape}")
print(f"LBP 64 features: {lbp_features64.shape}")
print(f"Color Histogram 64 features: {color_hist_features64.shape}")


Processed 200 images
Processed 400 images
Processed 600 images
Processed 800 images
Processed 1000 images
Processed 1200 images
Processed 1400 images
Processed 1600 images
Processed 1800 images
Processed 2000 images
Processed 2200 images
Processed 2400 images
Processed 2600 images
Processed 2800 images
Processed 3000 images
Processed 3200 images
Processed 3400 images
Processed 3600 images
Processed 3800 images
Processed 4000 images
Processed 4200 images
Processed 4400 images
Processed 4600 images
Processed 4800 images
Processed 5000 images
Processed 5200 images
Processed 5400 images
Processed 5600 images
Processed 5800 images
Processed 6000 images
Processed 6200 images
Processed 6400 images
Processed 6600 images
Processed 6800 images
Processed 7000 images
Processed 7200 images
Processed 7400 images
Processed 7600 images
Processed 7800 images
Processed 8000 images
Processed 8200 images
Processed 8400 images
Processed 8600 images
Processed 8800 images
Processed 9000 images
Processed 9200

# classifiers

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

def kmeans_clustering(features, num_clusters, max_iters=30, tol=1e-4):
    np.random.seed(40)
    centers = features[np.random.choice(features.shape[0], num_clusters, replace=False)]
    
    for i in range(max_iters):
        distances = np.linalg.norm(features[:, np.newaxis] - centers, axis=2)
        cluster_labels = np.argmin(distances, axis=1)
        
        new_centers = np.array([features[cluster_labels == k].mean(axis=0) for k in range(num_clusters)])
        if np.linalg.norm(new_centers - centers) < tol:
            break
        centers = new_centers
    
        print(f"step {i}")
    return cluster_labels, centers



def knn_predict(train_features, train_labels, test_features, k=10):
    predictions = []
    num = len(test_features)
    batch_size = 50  # Process test samples in batches for better memory utilization
    num_batches = (num + batch_size - 1) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, num)
        test_batch = test_features[start_idx:end_idx]
        
        distances = np.linalg.norm(train_features[:, None, :] - test_batch[None, :, :], axis=2)  # Shape: (train_size, batch_size)
        
        # Find nearest neighbors
        nearest_indices = np.argsort(distances, axis=0)[:k, :] 
        nearest_labels = train_labels[nearest_indices]  # Get the labels of the nearest neighbors
        
        # Determine the most common label for each test sample
        batch_predictions = [Counter(nearest_labels[:, col]).most_common(1)[0][0] for col in range(test_batch.shape[0])]
        predictions.extend(batch_predictions)
        
        print(f"Batch {batch_idx + 1}/{num_batches} completed")

    return np.array(predictions)



def train_svm(train_features, train_labels):
    svm = SVC(kernel="linear", C=1.0)
    svm.fit(train_features, train_labels)
    return svm






In [ ]:
from scipy.spatial.distance import cdist
from collections import Counter
import numpy as np

def assign_label_to_centers(centers, data, labels, num_closest=500):

    center_labels = {}

    distances = cdist(centers, data, metric='euclidean')  # Shape: (num_centers x num_samples)

    for center_idx, center_distances in enumerate(distances):
        closest_indices = np.argsort(center_distances)[:num_closest]
        closest_labels = labels[closest_indices]
        most_common_label, count = Counter(closest_labels).most_common(1)[0]
        center_labels[center_idx] = (most_common_label, count)

    return center_labels

import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score

def compute_test_accuracy(centers, center_labels, X_test, y_test):
    """
    Compute the prediction accuracy using cluster centers on the test set.

    Parameters:
    - centers (np.ndarray): Cluster centers (shape: num_clusters x num_features).
    - center_labels (dict): Dictionary mapping center indices to their labels.
    - X_test (np.ndarray): Test dataset (shape: num_test_samples x num_features).
    - y_test (np.ndarray): True labels for the test dataset (shape: num_test_samples).

    Returns:
    - accuracy (float): Accuracy of predictions on the test set.
    """
    predictions = []
    
    for test_sample in X_test:
        # Compute distances from test_sample to all cluster centers
        distances = np.linalg.norm(centers - test_sample, axis=1)
        
        # Find the index of the closest center
        closest_center_idx = np.argmin(distances)
        
        # Get the label of the closest center
        predicted_label = center_labels[closest_center_idx]
        predictions.append(predicted_label)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    return accuracy



# train

In [13]:

hog_features_array = np.array(hog_features)
lbp_features_array = np.array(lbp_features)
color_hist_features_array = np.array(color_hist_features)
lbp_features_array64 = np.array(lbp_features64)
color_hist_features_array64 = np.array(color_hist_features64)
labels_array = np.array(labels)

# Split Data into Training and Testing
X_train_hog, X_test_hog, y_train, y_test = train_test_split(hog_features_array, labels_array, test_size=0.2, random_state=42)
X_train_lbp, X_test_lbp, y_train_lbp, y_test_lbp= train_test_split(lbp_features_array, labels_array, test_size=0.2, random_state=42)
X_train_color, X_test_color, y_train_color , y_test_color = train_test_split(color_hist_features_array, labels_array, test_size=0.2, random_state=42)

X_train_lbp64, X_test_lbp64, y_train_lbp64, y_test_lbp64= train_test_split(lbp_features_array64, labels_array, test_size=0.2, random_state=42)
X_train_color64, X_test_color64, y_train_color64 , y_test_color64 = train_test_split(color_hist_features_array64, labels_array, test_size=0.2, random_state=42)



### HOG

In [14]:

print("Training SVM...")
start_time = time.time()
svm_model_hog = train_svm(X_train_hog, y_train)
svm_predictions_hog = svm_model_hog.predict(X_test_hog)
svm_accuracy_hog = accuracy_score(y_test, svm_predictions_hog)
print(f"SVM (HOG) Accuracy: {svm_accuracy_hog}, takiing: {time.time()-start_time} sec")

Training SVM...
SVM (HOG) Accuracy: 0.0995, takiing: 1980.5473673343658 sec


### Color hist

In [15]:

# # 1. K-Means Clustering
# print("Training K-Means for LBP...")
# start_time = time.time()
# num_clusters = len(np.unique(labels_array))

# kmeans_labels_color, kmeans_center_color = kmeans_clustering(X_train_color, num_clusters)

# print(f"K-Means (LBP) clustering complete, taking: {time.time() - start_time} sec")


# 2. KNN
print("Training K-Nearest Neighbors for color...")
start_time = time.time()
knn_predictions_color = knn_predict(X_train_color, y_train_color, X_test_color, k=10)
knn_accuracy_color = accuracy_score(y_test_color, knn_predictions_color)
print(f"K-Nearest Neighbors (color) Accuracy: {knn_accuracy_color}, taking: {time.time() - start_time} sec")

# 3. SVM
print("Training SVM for color...")
start_time = time.time()
svm_model_color = train_svm(X_train_color, y_train_color)
svm_predictions_color = svm_model_color.predict(X_test_color)
svm_accuracy_color= accuracy_score(y_test_color, svm_predictions_color)
print(f"SVM (color) Accuracy: {svm_accuracy_color}, taking: {time.time() - start_time} sec")


Training K-Nearest Neighbors for color...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Batch 39/200 completed
Batch 40/200 completed
Batch 41/200 completed
Batch 42/200 completed
B

### kmean


In [ ]:
# kmean
def kmeans_clustering(features, num_clusters, max_iters=30, tol=1e-4):
    """
    Performs K-Means clustering without relying on cluster labels.
    
    Args:
        features (np.ndarray): The input data to cluster.
        num_clusters (int): Number of clusters.
        max_iters (int): Maximum iterations for convergence.
        tol (float): Tolerance for convergence.

    Returns:
        centers (np.ndarray): The cluster centers.
    """
    np.random.seed(40)
    centers = features[np.random.choice(features.shape[0], num_clusters, replace=False)]
    
    for i in range(max_iters):
        # Calculate distances and assign points to the nearest center
        distances = np.linalg.norm(features[:, np.newaxis] - centers, axis=2)
        cluster_labels = np.argmin(distances, axis=1)
        
        # Update centers
        new_centers = np.array([features[cluster_labels == k].mean(axis=0) for k in range(num_clusters)])
        if np.linalg.norm(new_centers - centers) < tol:
            break
        centers = new_centers
        
        print(f"step {i}")
    
    return centers


def assign_labels_to_clusters(centers, features, labels, k=500):
    """
    Assigns a label to each cluster based on the most frequent label 
    among the closest `k` points to the center.

    Args:
        centers (np.ndarray): Cluster centers.
        features (np.ndarray): Input data.
        labels (np.ndarray): Corresponding labels for the input data.
        k (int): Number of closest points to consider.

    Returns:
        cluster_labels (dict): A dictionary mapping center indices to labels.
    """
    cluster_labels = {}
    
    for idx, center in enumerate(centers):
        # Compute distances to all features
        distances = np.linalg.norm(features - center, axis=1)
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels[nearest_indices]

        most_common_label = Counter(nearest_labels).most_common(1)[0][0]
        cluster_labels[idx] = most_common_label
    
    return cluster_labels


def predict_using_kmeans(centers, cluster_labels, test_features):
    """
    Predicts the labels for test data based on the closest cluster center.

    Args:
        centers (np.ndarray): Cluster centers.
        cluster_labels (dict): Mapping of cluster indices to labels.
        test_features (np.ndarray): Test data.

    Returns:
        predictions (np.ndarray): Predicted labels for the test data.
    """
    predictions = []
    
    for test_sample in test_features:
        # Find the nearest center
        distances = np.linalg.norm(centers - test_sample, axis=1)
        nearest_center_idx = np.argmin(distances)
        
        # Use the label of the nearest center
        predictions.append(cluster_labels[nearest_center_idx])
    
    return np.array(predictions)


num_clusters = len(np.unique(labels_array))

# Train K-Means without labels
centers = kmeans_clustering(X_train_color, num_clusters)

cluster_labels = assign_labels_to_clusters(centers, X_train_color, y_train_color, k=500)
kmeans_predictions = predict_using_kmeans(centers, cluster_labels, X_test_color)
kmeans_accuracy = accuracy_score(y_test_color, kmeans_predictions)
print(f"K-Means Accuracy: {kmeans_accuracy}")



### LBP

In [16]:

# # 1. K-Means Clustering
# print("Training K-Means for LBP...")
# start_time = time.time()
# num_clusters = len(np.unique(labels_array))
# kmeans_labels_lbp, _ = kmeans_clustering(X_train_lbp, num_clusters)
# knn_accuracy_lbp = accuracy_score(y_test_lbp, kmeans_labels_lbp)
# print(f"K-Means (LBP) clustering complete, taking: {time.time() - start_time} sec")


# 2. K-Nearest Neighbors
print("Training K-Nearest Neighbors for LBP...")
start_time = time.time()
knn_predictions_lbp = knn_predict(X_train_lbp, y_train_lbp, X_test_lbp, k=10)
knn_accuracy_lbp = accuracy_score(y_test_lbp, knn_predictions_lbp)
print(f"K-Nearest Neighbors (LBP) Accuracy: {knn_accuracy_lbp}, taking: {time.time() - start_time} sec")

# 3. Support Vector Machine (SVM)
print("Training SVM for LBP...")
start_time = time.time()
svm_model_lbp = train_svm(X_train_lbp, y_train_lbp)
svm_predictions_lbp = svm_model_lbp.predict(X_test_lbp)
svm_accuracy_lbp = accuracy_score(y_test_lbp, svm_predictions_lbp)
print(f"SVM (LBP) Accuracy: {svm_accuracy_lbp}, taking: {time.time() - start_time} sec")



Training K-Nearest Neighbors for LBP...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Batch 39/200 completed
Batch 40/200 completed
Batch 41/200 completed
Batch 42/200 completed
Bat

In [17]:

with open(f"m1acc","+w") as f:
    
    f.write(f"svm_hog acc {svm_accuracy_hog} \nsvm_lbp acc {svm_accuracy_lbp} \nknn_lbp acc  {knn_accuracy_lbp} ")
    f.write(f"knn_color acc {knn_accuracy_color} \nsvm_lbp acc {svm_accuracy_lbp} \nknn_lbp acc  {knn_accuracy_lbp} ")